In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

from src.recurrent_networks import RNN, BiNN, LSTM, GRU

## Sequence Forecasting with Recurrent Neural Networks

Step 1: Sequence Generation

In [ ]:
def generate_sequence(T=50, d=3, q=1):
    """Generate ONE (T,d) input and (T,q) output sequence"""
    t = np.linspace(0, 10, T)
    X = np.zeros((T, d))
    Y = np.zeros((T, q))
    
    # Create d input features
    for feature in range(d):
        freq = np.random.uniform(0.1, 0.5)
        phase = np.random.uniform(0, 2*np.pi)
        X[:, feature] = np.sin(2*np.pi*freq*t + phase) + 0.1*np.random.randn(T)
    
    # Create output (simple linear combination)
    Y[:, 0] = 0.5*X[:, 0] + 0.3*X[:, 1] - 0.2*X[:, 2]
    return torch.FloatTensor(X), torch.FloatTensor(Y)

In [ ]:
# Generate data
X_train, Y_train = generate_sequence(T=50, d=3, q=1)  # Single sequence
X_test, Y_test = generate_sequence(T=50, d=3, q=1)     # Another single sequence

print("Shapes:")
print(f"X_train: {X_train.shape} (should be (T,d))")
print(f"Y_train: {Y_train.shape} (should be (T,q))")

Step 2: Model Initialization and Training

In [ ]:
rnn = RNN(d=3, p=32, k=2, q=1)
binn = BiNN(d=3, p=32, k=2, q=1)
gru = GRU(d=3, p=32, k=2, q=1) 
lstm = LSTM(d=3, p=32, k=2, q=1)

In [ ]:
rnn_history= rnn.fit(X_train, Y_train, 
         alpha=0.1, epochs=250, verbose=False, decay="exp")
binn_history = binn.fit(X_train, Y_train, 
         alpha=0.1, epochs=250, verbose=False, decay="exp")
gru_history = gru.fit(X_train, Y_train, 
         alpha=0.1, epochs=250, verbose=False, decay="exp")
lstm_history = lstm.fit(X_train, Y_train, 
         alpha=0.1, epochs=250, verbose=False, decay="exp")

Step 3: Analyze training loss through epochs

In [ ]:
plt.figure(figsize=(20,5))

# plt.plot(rnn_history, marker='.', color='blue', label='Vanilla RNN')
# plt.plot(binn_history, marker='.', color='gold', label='BiNN')
plt.plot(gru_history, marker='.', color='red', label='GRU')
plt.plot(lstm_history, marker='.', color='green', label='LSTM')
plt.title('Training MSE Evolution')
plt.xlabel('Epochs')
plt.legend()
plt.show()

Step 4: Predict Outputs

In [ ]:
with torch.no_grad():
    Y_train_pred_gru = gru._forward_pass(X_train)
    Y_test_pred_gru = gru._forward_pass(X_test)
    Y_train_pred_lstm = lstm._forward_pass(X_train)
    Y_test_pred_lstm = lstm._forward_pass(X_test)

Step 5: Visualize Predictions

In [ ]:
plt.figure(figsize=(15, 10))

# Training results
plt.subplot(2, 1, 1)
plt.plot(Y_train[:, 0], color='black', label='True')
plt.plot(Y_train_pred_gru[:, 0], '--', color='red', label='GRU')
plt.plot(Y_train_pred_lstm[:, 0], '--', color='green', label='LSTM')
plt.title(f'Training Sequence\n(X: {X_train.shape}, Y: {Y_train.shape})')
plt.xlabel('Timesteps')
plt.legend()

# Test results
plt.subplot(2, 1, 2)
plt.plot(Y_test[:, 0], color='black', label='True')
plt.plot(Y_test_pred_gru[:, 0], '--', color='red', label='GRU')
plt.plot(Y_test_pred_lstm[:, 0], '--', color='green', label='LSTM')
plt.title(f'Test Sequence\n(X: {X_test.shape}, Y: {Y_test.shape})')
plt.xlabel('Timesteps')
plt.legend()

plt.tight_layout()
plt.show()


Step 6: Evaluate MSE Loss

In [ ]:
train_mse_gru = torch.mean((Y_train_pred_gru - Y_train)**2).item()
test_mse_gru = torch.mean((Y_test_pred_gru - Y_test)**2).item()
print(f'\nGRU:\nTrain: {train_mse_gru:.5f}\nTest: {test_mse_gru:.5f}')

train_mse_lstm = torch.mean((Y_train_pred_lstm - Y_train)**2).item()
test_mse_lstm = torch.mean((Y_test_pred_lstm - Y_test)**2).item()
print(f'\nLSTM:\nTrain: {train_mse_lstm:.5f}\nTest: {test_mse_lstm:.5f}')